developed by Chrisowalandis Deligio, Sebastian Gampe

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import random
import os
import numpy as np
from cnt.model import (DesignEstimator, RelationExtractor, save_pipeline, load_pipeline, predict_re_single_sentence, 
relations_from_adjectives_df, relations_from_adjectives_single, concat_relations)
from cnt.annotate import (annotate, annotate_single_design, 
                          annotate_designs, 
                          extract_string_from_annotation, labeling_eng)
from cnt.extract_relation import (path, NERTransformer, FeatureExtractor)
from cnt.evaluate import Metrics
from cnt.vectorize import (Doc2Str, Path2Str, Verbs2Str, AveragedPath2Vec, 
                           AveragedRest2Vec)
from cnt.io import (replace_left_right)
from cnt.io import  Database_Connection
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Database access in text file: "mysql+mysqlconnector://root:YourConnection" -> Format user:password@IP/Database
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)

In [3]:
id_col = "id"
design_col = "design_en"

In [4]:
import yaml
import_path = "../data/English_RE_data_new.yaml"
with open(import_path, encoding='utf8') as f:
    dictionary = yaml.safe_load(f)
    d = {replace_left_right(key): value for key, value in dictionary.items()}

In [5]:
relation_counts = {}
labels = []
for sentence, relations in d.items():
    for rel in relations:
        rel_name = rel[1]
        if rel_name not in relation_counts:
            relation_counts[rel_name] = 1
        else:
            relation_counts[rel_name] += 1

sorted(relation_counts.items(), key= lambda x: (-x[1], x[0]))

[('holding', 1038),
 ('wearing', 749),
 ('resting_on', 207),
 ('seated_on', 74),
 ('standing', 33),
 ('grasping', 32),
 ('crowning', 10),
 ('feeding', 10),
 ('coiling', 6),
 ('breaking', 4),
 ('flying_over', 2),
 ('receiving', 2),
 ('escorted_by', 1)]

In [6]:
language = "_en"
add_columns = ["name"+language, "alternativenames"+language] #["id", "name"+language, "alternativenames"+language]

In [7]:
entities = {
    "PERSON": dc.load_entities_from_db("nlp_list_person", ["name", "alternativenames"], ["alternativenames"], ",", True),
    "OBJECT": dc.load_entities_from_db("nlp_list_obj", add_columns, [add_columns[1]], ",", True),
    "ANIMAL": dc.load_entities_from_db("nlp_list_animal", add_columns, [add_columns[1]], ",", True),
    "PLANT": dc.load_entities_from_db("nlp_list_plant", add_columns, [add_columns[1]], ",", True)}

In [29]:
X_list, y_list = labeling_eng(d, entities)

In [30]:
X = pd.DataFrame({design_col: X_list, "y" : y_list})

In [31]:
X.shape

(1029, 2)

In [32]:
X[id_col] = X.index

In [33]:
X.style

,design_en,y,id
0,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]",0
1,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('Amphora', 'OBJECT', 'holding', 'poppy', 'PLANT'), ('Amphora', 'OBJECT', 'holding', 'ears of corn', 'PLANT')]",1
2,"Anubis advancing right, holding palm branch with right hand over shoulder; in field to left monogram; to right emblem of two conjoined foreparts of prancing horses to right and left. Short ground line.","[('Anubis', 'PERSON', 'holding', 'palm branch', 'OBJECT')]",2
3,"Aphrodite standing facing, head right, wearing double chiton and epiblema, holding a long sceptre in right and an apple in outstretched left hand. Border of dots.","[('Aphrodite', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aphrodite', 'PERSON', 'holding', 'sceptre', 'OBJECT')]",3
4,"Apollo (Bonus Eventus) standing facing, head left, holding patera in outstretched right hand over lighted altar and laurel branch in left hand.","[('Apollo', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",4
5,"Apollo (Sauroktonos) standing right, legs crossed, holding arrow in right hand; resting left arm tree stump with a lizard climbing up.","[('Apollo', 'PERSON', 'holding', 'arrow', 'OBJECT')]",5
6,"Apollo and Artemis facing each other and clasping hands; to left, Apollo standing right, seen from behind, holding laurel branch in lowered left hand; to right Artemis, standing left, wearing a short chiton and boots, holding a bow in left arm; lighted altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",6
7,"Apollo and Artemis in front of large conical torch; to left, Apollo standing right, seen from behind, holding spear in left arm and chlamys over left shoulder clasping right hands with Artemis, to right, standing left, wearing a short chiton and boots, holding a bow in left hand.","[('Apollo', 'PERSON', 'holding', 'spear', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'chlamys', 'OBJECT'), ('Apollo', 'PERSON', 'grasping', 'Artemis', 'PERSON'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",7
8,"Apollo and Artemis, seen from front each other, clasping right hands; to left, Apollo, right, holding laurel branch; to right, Artemis, left, holding bow; altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",8
9,"Apollo enthroned left, holding laurel branch in outstreched and lowered right hand and lyra set down behind him in left hand.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",9


In [34]:
for index, row in X.iterrows():
    if "'" in row["design_en"]:
        X.at[index, "design_en"] = row["design_en"].replace("'", "\\'")

In [35]:
X.style

,design_en,y,id
0,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]",0
1,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('Amphora', 'OBJECT', 'holding', 'poppy', 'PLANT'), ('Amphora', 'OBJECT', 'holding', 'ears of corn', 'PLANT')]",1
2,"Anubis advancing right, holding palm branch with right hand over shoulder; in field to left monogram; to right emblem of two conjoined foreparts of prancing horses to right and left. Short ground line.","[('Anubis', 'PERSON', 'holding', 'palm branch', 'OBJECT')]",2
3,"Aphrodite standing facing, head right, wearing double chiton and epiblema, holding a long sceptre in right and an apple in outstretched left hand. Border of dots.","[('Aphrodite', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aphrodite', 'PERSON', 'holding', 'sceptre', 'OBJECT')]",3
4,"Apollo (Bonus Eventus) standing facing, head left, holding patera in outstretched right hand over lighted altar and laurel branch in left hand.","[('Apollo', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",4
5,"Apollo (Sauroktonos) standing right, legs crossed, holding arrow in right hand; resting left arm tree stump with a lizard climbing up.","[('Apollo', 'PERSON', 'holding', 'arrow', 'OBJECT')]",5
6,"Apollo and Artemis facing each other and clasping hands; to left, Apollo standing right, seen from behind, holding laurel branch in lowered left hand; to right Artemis, standing left, wearing a short chiton and boots, holding a bow in left arm; lighted altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",6
7,"Apollo and Artemis in front of large conical torch; to left, Apollo standing right, seen from behind, holding spear in left arm and chlamys over left shoulder clasping right hands with Artemis, to right, standing left, wearing a short chiton and boots, holding a bow in left hand.","[('Apollo', 'PERSON', 'holding', 'spear', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'chlamys', 'OBJECT'), ('Apollo', 'PERSON', 'grasping', 'Artemis', 'PERSON'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",7
8,"Apollo and Artemis, seen from front each other, clasping right hands; to left, Apollo, right, holding laurel branch; to right, Artemis, left, holding bow; altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",8
9,"Apollo enthroned left, holding laurel branch in outstreched and lowered right hand and lyra set down behind him in left hand.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",9


In [36]:
def get_id(design):
    try:
        return (dc.create_own_query("select id from nlp_training_designs where design_en='"+design+"';").id)[0]
    except:
        return "Null"

In [37]:
%%capture
X["db_id"] = X.apply(lambda row: get_id(row.design_en), axis=1)

In [38]:
X.style

,design_en,y,id,db_id
0,"Aequitas standing left, wearing chiton and himation, holding scales and cornucopia.","[('Aequitas', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aequitas', 'PERSON', 'wearing', 'himation', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'scales', 'OBJECT'), ('Aequitas', 'PERSON', 'holding', 'cornucopia', 'OBJECT')]",0,6652
1,Amphora with ribbed surface and crooked handles containing two ears of corn and poppy.,"[('Amphora', 'OBJECT', 'holding', 'poppy', 'PLANT'), ('Amphora', 'OBJECT', 'holding', 'ears of corn', 'PLANT')]",1,9
2,"Anubis advancing right, holding palm branch with right hand over shoulder; in field to left monogram; to right emblem of two conjoined foreparts of prancing horses to right and left. Short ground line.","[('Anubis', 'PERSON', 'holding', 'palm branch', 'OBJECT')]",2,1856
3,"Aphrodite standing facing, head right, wearing double chiton and epiblema, holding a long sceptre in right and an apple in outstretched left hand. Border of dots.","[('Aphrodite', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Aphrodite', 'PERSON', 'holding', 'sceptre', 'OBJECT')]",3,1582
4,"Apollo (Bonus Eventus) standing facing, head left, holding patera in outstretched right hand over lighted altar and laurel branch in left hand.","[('Apollo', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Bonus Eventus', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",4,24725
5,"Apollo (Sauroktonos) standing right, legs crossed, holding arrow in right hand; resting left arm tree stump with a lizard climbing up.","[('Apollo', 'PERSON', 'holding', 'arrow', 'OBJECT')]",5,6761
6,"Apollo and Artemis facing each other and clasping hands; to left, Apollo standing right, seen from behind, holding laurel branch in lowered left hand; to right Artemis, standing left, wearing a short chiton and boots, holding a bow in left arm; lighted altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",6,24788
7,"Apollo and Artemis in front of large conical torch; to left, Apollo standing right, seen from behind, holding spear in left arm and chlamys over left shoulder clasping right hands with Artemis, to right, standing left, wearing a short chiton and boots, holding a bow in left hand.","[('Apollo', 'PERSON', 'holding', 'spear', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'chlamys', 'OBJECT'), ('Apollo', 'PERSON', 'grasping', 'Artemis', 'PERSON'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",7,67
8,"Apollo and Artemis, seen from front each other, clasping right hands; to left, Apollo, right, holding laurel branch; to right, Artemis, left, holding bow; altar between them.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT'), ('Artemis', 'PERSON', 'holding', 'bow', 'OBJECT')]",8,24787
9,"Apollo enthroned left, holding laurel branch in outstreched and lowered right hand and lyra set down behind him in left hand.","[('Apollo', 'PERSON', 'holding', 'laurel branch', 'OBJECT')]",9,24723


In [39]:
X[X['db_id'] == 'Null'].style

,design_en,y,id,db_id
23,"Nude Apollo standing left, holding patera and two arrows.","[('Apollo', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Apollo', 'PERSON', 'holding', 'arrows', 'OBJECT')]",23,Null
85,"At left Artemis standing right, wearing kalathos, short chiton and boots, clasping right hands with city goddess, at right, standing facing, head left.","[('Artemis', 'PERSON', 'wearing', 'kalathos', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'chiton', 'OBJECT'), ('Artemis', 'PERSON', 'wearing', 'boots', 'OBJECT')]",85,Null
89,"At left, emperor (Septimius Severus) standing facing, head right, wearing helmet, cuirass and boots, holding spear in right hand and parazonium in left arm; to right, Nike standing left, holding palm branch in left arm, crowning with laurel wreath in right outstretched arm the emperor. Ground line. Border of dots.","[('Septimius Severus', 'PERSON', 'wearing', 'helmet', 'OBJECT'), ('Septimius Severus', 'PERSON', 'wearing', 'cuirass', 'OBJECT'), ('Septimius Severus', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Septimius Severus', 'PERSON', 'holding', 'spear', 'OBJECT'), ('Septimius Severus', 'PERSON', 'holding', 'parazonium', 'OBJECT'), ('Nike', 'PERSON', 'holding', 'palm branch', 'OBJECT')]",89,Null
90,"At left, laureate and cuirassed emperor (Caracalla) standing left, wearing paludamentum and boots, holding patera and parazonium; altar at his feet; to right, Nike standing left, crowning the emperor with laurel wreath in right outstretched arm, holding palm branch in left arm.","[('Caracalla', 'PERSON', 'wearing', 'paludamentum', 'OBJECT'), ('Caracalla', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('Caracalla', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Caracalla', 'PERSON', 'holding', 'parazonium', 'OBJECT'), ('Nike', 'PERSON', 'crowning', 'emperor', 'PERSON'), ('Nike', 'PERSON', 'holding', 'palm branch', 'OBJECT')]",90,Null
109,"Athena enthroned left; throne decorated with Sphinx, left, front leg ends in lion\'s paw; holding in right hand patera from which she feeds a serpent entwined around tree in front of her, leaning left arm on throne back; behind her, owl sitting left on a frontal shield.","[('Athena', 'PERSON', 'holding', 'patera', 'OBJECT'), ('Athena', 'PERSON', 'feeding', 'serpent', 'ANIMAL'), ('Athena', 'PERSON', 'resting_on', 'throne', 'OBJECT'), ('owl', 'ANIMAL', 'seated_on', 'shield', 'OBJECT')]",109,Null
212,"City goddess standing right, wearing kalathos, clasping right hands with laureate emperor standing left, wearing cuirass and boots.","[('City goddess', 'PERSON', 'wearing', 'kalathos', 'OBJECT'), ('City goddess', 'PERSON', 'wearing', 'cuirass', 'OBJECT'), ('City goddess', 'PERSON', 'wearing', 'boots', 'OBJECT'), ('emperor', 'PERSON', 'wearing', 'cuirass', 'OBJECT'), ('emperor', 'PERSON', 'wearing', 'boots', 'OBJECT')]",212,Null
225,"Confronting busts of Philippus, wearing paludamentum and of Serapis, draped and wearing kalathos.","[('Serapis', 'PERSON', 'wearing', 'kalathos', 'OBJECT')]",225,Null
236,Cybele seated left on lion walking right.,"[('Cybele', 'PERSON', 'seated_on', 'lion', 'ANIMAL')]",236,Null
305,"Draped bust of bearded Serapis, right, wearing kalathos.","[('Serapis', 'PERSON', 'wearing', 'kalathos', 'OBJECT')]",305,Null
315,"Eagle standing facing, head left, wings spread, holding wreath in beak. Border of dots.","[('Eagle', 'ANIMAL', 'holding', 'wreath', 'OBJECT')]",315,Null


In [20]:
X.shape

(1029, 4)

In [21]:
X = X.loc[X.db_id!="Null"]

In [22]:
X.shape

(996, 4)

In [23]:
X.head(5)

,design_en,y,id,db_id
0,"Aequitas standing left, wearing chiton and him...","[(Aequitas, PERSON, wearing, chiton, OBJECT), ...",0,6652
1,Amphora with ribbed surface and crooked handle...,"[(Amphora, OBJECT, holding, poppy, PLANT), (Am...",1,9
2,"Anubis advancing right, holding palm branch wi...","[(Anubis, PERSON, holding, palm branch, OBJECT)]",2,1856
3,"Aphrodite standing facing, head right, wearing...","[(Aphrodite, PERSON, wearing, chiton, OBJECT),...",3,1582
4,"Apollo (Bonus Eventus) standing facing, head l...","[(Apollo, PERSON, holding, patera, OBJECT), (A...",4,24725


In [24]:
entities = dc.create_own_query("select * from nlp_list_entities;")

In [25]:
entities[["id", "name_en", "alternativenames_en"]]

,id,name_en,alternativenames_en
0,1,Abundantia,None
1,2,Actaeon,None
2,3,Aemilian,None
3,4,Aeneas,None
4,5,Aequitas,Equitas
...,...,...,...
844,845,turning,None
845,846,sailing,None
846,847,escorted_by,escorted by
847,848,wearing,None


In [26]:
ent_dict = {}
for index, row in entities[["id", "name_en", "alternativenames_en"]].iterrows():
    ent_dict[row["id"]] = (row["name_en"]+", "+row["alternativenames_en"]).replace(", None","")#.lower()

In [30]:
result = pd.DataFrame(columns=["design_id", "subject", "subject_str", "predicate", "predicate_str", "object", "object_str"])

In [31]:
def get_id(ent):
    for row in ent_dict:
        for entry in ent_dict[row].split(', '):
            if ent == entry:
                return row
            if ent.lower() == entry:
                return row
    return ent

In [32]:
for index, row in X.iterrows():
    for y in row.y:
        subject = get_id(y[0])
        subject_str = y[0]
        predicate = get_id(y[2])
        predicate_str = y[2]
        obj = get_id(y[3])
        obj_str = y[3]
        if type(subject) == int and type(predicate) == int and type(obj) == int:
            result = result._append({"design_id": row.db_id, 
                                     "subject": subject, 
                                     "subject_str": subject_str,
                                     "predicate": predicate, 
                                     "predicate_str": predicate_str,
                                     "object": obj,
                                     "object_str": obj_str}, ignore_index=True)
        else:
            print(subject, predicate, obj)

In [33]:
result.style

,design_id,subject,subject_str,predicate,predicate_str,object,object_str
0,6652,5,Aequitas,848,wearing,466,chiton
1,6652,5,Aequitas,848,wearing,526,himation
2,6652,5,Aequitas,826,holding,608,scales
3,6652,5,Aequitas,826,holding,484,cornucopia
4,9,419,Amphora,826,holding,815,poppy
5,9,419,Amphora,826,holding,799,ears of corn
6,1856,26,Anubis,826,holding,568,palm branch
7,1582,27,Aphrodite,848,wearing,466,chiton
8,1582,27,Aphrodite,826,holding,609,sceptre
9,24725,29,Apollo,826,holding,571,patera


In [34]:
result.shape

(2090, 7)

In [35]:
result.to_sql("nlp_relation_extraction_en_v5",dc.mysql_connection,if_exists="append", index=False)

2090